In [1]:
!pip install ultralytics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ultralytics import YOLO

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import time
import torch
import cvzone
import requests # requests kütüphanesini ekleyin

# --- Backend Entegrasyon Bilgileri ---
BACKEND_URL = "http://localhost:8080/api/tracking/status" # Spring Boot uygulamanızın çalıştığı adres
user_id = 1 # Takip edilecek kullanıcının ID'si. Bu değeri giriş yapan kullanıcıya göre ayarlamalısınız.
# -------------------------------------

capture = cv.VideoCapture("video.mp4")
# ... (Diğer kurulum kodları)

model1 = YOLO("best.pt")

prev_time = 0
last_status = False # Son gönderilen durumu takip etmek için
status_send_interval = 5 # Her 5 saniyede bir durumu göndermek için (saniye)
last_send_time = time.time() # Son gönderme zamanını tutmak için

while True:
    isTrue, frame = capture.read()
    frame = cv.resize(frame, (1280, 720))
    results = model1(frame,stream=True, verbose=False)
    names = model1.names

    # Yatma durumu kontrolü için bir bayrak
    is_lying_down_in_frame = False

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1,y1,x2,y2 = box.xyxy[0]
            x1,y1,x2,y2 = int(x1), int(y1), int(x2), int(y2)

            cv.rectangle(frame,(x1,y1),(x2,y2),(255,0,255),1)
            conf = round(float(box.conf[0]),2)
            className = names[int(box.cls)]

            cvzone.putTextRect(frame,f'{className} {conf}',
                         (max(0,x1), max(35,y1)), scale=0.5, thickness=1, offset=4)

            # Algılanan sınıf 'lying' ise durumu güncelleyin
            if className == 'lying': # Modelinizin verdiği sınıf adı 'lying' veya benzeri olmalı
                is_lying_down_in_frame = True


    # --- Spring Boot Entegrasyonu: Durum Güncellemesi ---
    current_time = time.time()
    # Belirli aralıklarla (örneğin 5 saniyede bir) veya durum değiştiğinde gönderin
    if current_time - last_send_time >= status_send_interval or is_lying_down_in_frame != last_status:
        try:
            # Gönderilecek veri StatusUpdateRequest DTO'suna uymalı
            payload = {
                "userId": user_id,
                "isLyingDown": is_lying_down_in_frame
            }
            # POST isteği gönder
            response = requests.post(BACKEND_URL, json=payload)
            response.raise_for_status() # Hata durumunda istisna fırlatır

            print(f"Durum başarıyla gönderildi: userId={user_id}, isLyingDown={is_lying_down_in_frame}")

            last_status = is_lying_down_in_frame
            last_send_time = current_time # Gönderme zamanını güncelleyin

        except requests.exceptions.RequestException as e:
            print(f"Hata oluştu, durum gönderilemedi: {e}")

    # ... (FPS hesaplama ve görüntüleme kodları)
    curr_time = time.time()
    fps = 1 / (curr_time - prev_time) if prev_time else 0
    prev_time = curr_time
    cv.putText(frame, f'FPS: {int(fps)}', (20, 40),
               cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    if cv.waitKey(1) & 0xFF == ord("q"):
        break

    cv.imshow("Video", frame)

capture.release()
cv.destroyAllWindows()

Durum başarıyla gönderildi: userId=1, isLyingDown=False
Durum başarıyla gönderildi: userId=1, isLyingDown=False
Durum başarıyla gönderildi: userId=1, isLyingDown=False


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
